In [8]:
import pandas as pd
import re
import json
import csv

from gpt_prompt import GPTClient

In [9]:
def extract_gpt_snippets(response_text):
    # Extract the result section
    result_section = re.search(r'<result>(.*)</result>', response_text, re.DOTALL).group(1)
    # Split the result section into individual code snippets
    code_snippets = result_section.strip().split('--\n')

    return code_snippets

def prompt_user_feedback(snippets):
    """
    Prompts a user via the standard input to ask if he is satisfied with the generated snippets.
    
    If the user types in "y", "yes" or just presses enter, 
        the function returns (True, []) tuple
    If instead the user types in the comma-separated list of digits like "1,2,4-6,9", 
        the function returns (False, [1, 2, 4, 5, 6, 9]) tuple
    """
    # Step 1: Print each snippet with a number and separator
    for i, snippet in enumerate(snippets, start=1):
        print(f"{i}.\n{snippet.strip()}")
        print('-' * 20)  # Separator for better readability

    # Step 2: Prompt user for feedback
    user_input = input("Are you satisfied with the generated snippets? (y/yes/n/no): ").strip().lower()

    # Step 3: Check user input for satisfaction
    if user_input in ['y', 'yes']:
        return True, []

    # Step 4: Check if user wants to regenerate any of the samples
    user_input = input("Do you want to regenerate any of the snippets? If not, type n/no or hit Enter, " + \
                       "otherwise specify a comma-separated list of values (e.g. 1,2,4-6,9)").strip().lower()
    if user_input in ['', 'n', 'no']:
        return False, []
        
    try:
        numbers = []
        for part in user_input.split(','):
            if '-' in part:
                start, end = map(int, part.split('-'))
                numbers.extend(range(start, end + 1))
            else:
                numbers.append(int(part))
        
        # Validate numbers as indices of the snippets list
        valid_indices = range(1, len(snippets) + 1)
        print(numbers, valid_indices)
        invalid_numbers = [num for num in numbers if num not in valid_indices]

        if invalid_numbers:
            print(f"Invalid snippet numbers: {invalid_numbers}. Please provide valid indices.")
            return prompt_user_feedback(snippets)  # Retry prompting if input is invalid
        
        return False, sorted(set(numbers))  # Return unique sorted list of numbers
    except ValueError:
        print("Invalid input format. Please provide either 'y', 'yes', or a comma-separated list of numbers.")
        return prompt_user_feedback(snippets)  # Retry prompting if input is invalid

In [10]:
params = {
    "model": 'gpt-4-turbo-2024-04-09',
    "temperature": 1,
    "verbose": True
}

In [11]:
# CSV output file path
output_fname = 'result_gpt.csv'
input_fname  = 'dataset_correct.csv'

def save_snippets(style, snippets, label, processed_snippets_ids):
    with open(output_fname, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        for i, snippet in enumerate(snippets):
            is_processed = i+1 in processed_snippets_ids
            csv_writer.writerow([style, snippet, label, is_processed])

In [12]:
df = pd.read_csv(input_fname)
df

,style,code,aspects,label,processed
0,Aligned with opening delimiter.,"foo = long_function_name(var_one, var_two,\r\n...",n/fs,correct,True
1,Add 4 spaces (an extra level of indentation) t...,"def long_function_name(\r\n var_one, va...",n/fs,correct,True
2,Hanging indents should add a level.,"foo = long_function_name(\r\n var_one, var_...",n/fs,correct,True
3,No extra indentation (same level of indent).,if (this_is_one_thing and\r\n that_is_anoth...,n,correct,True
4,Add some extra indentation on the conditional ...,if (this_is_one_thing\r\n and that_is_a...,n,correct,True
5,The closing brace/bracket/parenthesis on multi...,"my_list = [\r\n 1, 2, 3,\r\n 4, 5, 6,\r\n]",n/cl,correct,True
6,The closing brace/bracket/parenthesis on multi...,result = some_function_that_takes_arguments(\r...,n/fs,correct,True


In [13]:
# Discard all the processed examples
unprocessed_df = df[~df['processed']]
unprocessed_df

,style,code,aspects,label,processed


In [14]:
client = GPTClient()

for i, sample in unprocessed_df.iterrows():
    print(f'Sample #{i}: {sample.style}', end='\n' * 2)
    
    # Parse the aspects as the list of abbreviations that generate_correct_samples() function expects
    aspects = sample.aspects.split('/')

    # Prompt GPT and get its generated snippets based on the example from the sample row in the dataset.csv dataframe
    response = client.generate_correct_samples(prompt_rule=sample.style, prompt_code=sample.code,
                                               aspects_list=aspects, **params)
    response_text = response.choices[0].message.content
    if params['verbose']:
        print(response_text)
    
    code_snippets = extract_gpt_snippets(response_text)
    
    # Prompt user if he is satisfied with the generated snippets
    processed_answer, snippets_ids_to_regenerate = prompt_user_feedback(code_snippets)
    
    # If not, ask GPT to regenerate wrong snippets
    while(snippets_ids_to_regenerate):
        snippets_to_regenerate = [code_snippets[i-1] for i in snippets_ids_to_regenerate]
        print(snippets_to_regenerate)
        
        new_snippets_response = client.regenerate_correct_samples(snippets_to_regenerate, **params)
        new_snippets = extract_gpt_snippets(new_snippets_response.choices[0].message.content)

        for i, new_snippet in zip(snippets_ids_to_regenerate, new_snippets):
            code_snippets[i-1] = new_snippet

        last_snippets = snippets_ids_to_regenerate
        processed_answer, snippets_ids_to_regenerate = prompt_user_feedback(code_snippets)

    # mark last_snippets as unprocessed if the user is still dissatisfied with them:
    if processed_answer == False:
        processed_snippets_ids = [i for i in range(1, 10 + 1) if i not in last_snippets]
    else:
        processed_snippets_ids = [i for i in range(1, 10 + 1)]
    save_snippets(sample.style, code_snippets, sample.label, processed_snippets_ids)
    
    df.loc[i, 'processed'] = processed_answer
    client.clear_history()

In [15]:
# Mark the processed samples
df.to_csv(input_fname, mode='w', index=False, quoting=csv.QUOTE_ALL)